In [2]:
from camel_tools.utils import normalize

import re
import torch
import re
from camel_tools.utils import normalize
from ibm_watsonx_ai.foundation_models import Model
import numpy as np
from camel_tools.utils import normalize
from pyarabic.araby import strip_tashkeel, strip_tatweel
import camel_tools.utils.normalize as normalize
from camel_tools.tokenizers.word import simple_word_tokenize
import chromadb
import random
import string


In [3]:
import os
def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : os.environ.get("APIKEY")
		}

In [4]:
project_id =os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


In [5]:
model_1_id = "sdaia/allam-1-13b-instruct"
model_2_id = "google/flan-ul2"

In [6]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 1500,
    "repetition_penalty": 1.1
}

In [7]:
from ibm_watsonx_ai.foundation_models import Model

model_1 = Model(
	model_id = model_1_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)

model_2 = Model(
	model_id = model_2_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)

In [8]:
filename = 'wasl.txt'
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

lessonInfo = "الوحدة الأولى الخلفاء الراشدون / الدرس الاول الخلفاء الراشدون - الدراسات الاجتماعية - خامس ابتدائي"

In [9]:
def preprocess_arabic_text(text):
    text = strip_tashkeel(text)
    text = strip_tatweel(text)
    text = normalize.normalize_alef_ar(text)        
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'@#$٪^&*[{}[\]""",]', '', text)
    text = re.sub(r'[A-Za-z]', '', text)
    text =simple_word_tokenize(text)
    text= ' '.join(text)
    return text
    

In [10]:
preprocessedText= preprocess_arabic_text(text)

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

documents = [Document(page_content=preprocessedText)]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=100)


texts = text_splitter.split_documents(documents)
print(texts)

[Document(page_content='السلام عليكم ورحمة الله وبركاته اعزائي طلاب وطالبات الصف الخامس اهلا بكم من جديد في درس من دروس مادتنا الدراسات الاجتماعية معي معلمتكم الاستاذ العنود الاعتيبي برفقة زميلنا الاستاذ عماد الذويب في لغة الاشارة فاهلا بكم بسم الله نبدا درسنا الثاني الوحدة الاولى الخلفاء الراشدون ماذا سوف اتعلم في هذه الوحدة يا ترى صفات الخلفاء الراشدين اعمال الخلفاء الراشدين نشر الاسلام في عهد الخلفاء الراشدين نتائج الفتن بداية درس الاولى الخلفاء الراشدين تليها دروس'), Document(page_content='نشر الاسلام في عهد الخلفاء الراشدين نتائج الفتن بداية درس الاولى الخلفاء الراشدين تليها دروس الوحدة الخليفة ابو بكر الصديق الخليفة عمر بن الخطاب الخليفة عثمان بن عفاق الخليفة علي بن ابي طالب في نهاية كل وحدة يوجد لدينا تقويم شامل للوحدة محاور درسنا لهذا اليوم الدرس الاول من دروس مادة الدراسات الاجتماعية الخلفاء الراشدين صفات الخلفاء الراشدين اعمال الخلفاء الراشدين تسلسل الخط الزمني للخلافة الراشدة اهم معارك الخلفاء الراشدين مع الروم والفرس هذه'), Document(page_content='الخلفاء الراشدين تسلسل الخط

In [27]:
from transformers import BertModel, BertTokenizer

# Load the pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def generate_embeddings(texts):
    vector_embeddings = []
    vector_documents = []
    vector_metadatas = []
    vector_ids = []

    for i, text in enumerate(texts):
        # Encode the text using the BERT tokenizer
        input_ids = bert_tokenizer.encode(text.page_content, return_tensors='pt')

        with torch.no_grad():
            output = bert_model(input_ids)[0]
            embedding = output[:, 0, :]  # Take the embedding of the [CLS] token

        vector_embeddings.append(embedding.squeeze().tolist())
        vector_documents.append(text.page_content)
        metadata = {"chunk_number": i, "text_length": len(text.page_content)}
        vector_metadatas.append(metadata)
        random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        id = "{}-{}".format(i, random_string)
        vector_ids.append(id)
        print("Document {} has been embedded.".format(i))

    return vector_embeddings, vector_documents, vector_metadatas, vector_ids

def hydrate_chromadb(texts):
    chroma_client = chromadb.Client()

    # Create a new collection (or replace the existing one)
    collection_name = "my_collection"
    try:
        collection = chroma_client.delete_collection(name=collection_name)
    except:
        print("Collection didn't exist - nothing to do.")
    collection = chroma_client.create_collection(name=collection_name)

    # Generate embeddings and organize documents and metadata
    vector_embeddings, vector_documents, vector_metadatas, vector_ids = generate_embeddings(texts)

    # Add the vectors, documents, and metadata to the ChromaDB collection
    collection.add(
        embeddings=vector_embeddings,
        documents=vector_documents,
        metadatas=vector_metadatas,
        ids=vector_ids
    )
    print("ChromaDB collection has been hydrated.")
    return collection
# Hydrate ChromaDB with the data from `texts`
chroma_collection = hydrate_chromadb(texts)
print(type(chroma_collection))



A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Collection didn't exist - nothing to do.
Document 0 has been embedded.
Document 1 has been embedded.
Document 2 has been embedded.
Document 3 has been embedded.
Document 4 has been embedded.
Document 5 has been embedded.
Document 6 has been embedded.
Document 7 has been embedded.
Document 8 has been embedded.
Document 9 has been embedded.
Document 10 has been embedded.
Document 11 has been embedded.
Document 12 has been embedded.
Document 13 has been embedded.
Document 14 has been embedded.
Document 15 has been embedded.
Document 16 has been embedded.
Document 17 has been embedded.
Document 18 has been embedded.
Document 19 has been embedded.
Document 20 has been embedded.
Document 21 has been embedded.
Document 22 has been embedded.
Document 23 has been embedded.
Document 24 has been embedded.
Document 25 has been embedded.
Document 26 has been embedded.
Document 27 has been embedded.
Document 28 has been embedded.
Document 29 has been embedded.
Document 30 has been embedded.
Document

In [30]:
def proximity_search(question):
    # Encode the question using the BERT tokenizer
    input_ids = bert_tokenizer.encode(question, return_tensors='pt')

    # Pass the question through the BERT model to get the embedding
    with torch.no_grad():
        output = bert_model(input_ids)[0]
        query_vectors = output[:, 0, :]  # Take the embedding of the [CLS] token

    query_result = chroma_collection.query(
        query_embeddings=query_vectors.squeeze().tolist(),
        n_results=3,
        include=["documents", "metadatas", "distances"]
    )
    documents = list(reversed(query_result["documents"][0]))
    return "\n".join(documents)

In [19]:
prompt_input1="""انت مساعد ذكاء اصطناعي ومهمتك هي استخدام النص المرفق ادناه للرد على الاسئلة المطروحة"""
prompt_input2="""
قم بفهم سياق النص ومقارنته جيدا بالسؤال للخروج باجابة صحيحة للسؤال من النص المعطى.
يجب أن يكون الرد صحيح ومبني على المعلومات والحقائق الموجودة في النص المعطى.
استخدم قدرات علام في تحسين جودة النص المخرج ليكون اوضح واسهل
لاتقم بالاجابة على اي سؤال خارج سياق النص المعطى.
لاتقم بتغيير السؤال ابدا .
لاتقم بالاجابة الا على الاسئلة المتعلقة بالنص المعطى فقط.
لاتقم بالاجابة على الاسئلة خارج عنوان المادة والدرس ايا كانت.
انت اداة لمساعدة الطلاب في فهم النصوص والاجابة على الاسئلة المتعلقة بها.
لاتقم بتغيير السؤال .
اذا كانت الاجابة غير موجودة قم بالرد ب: لايمكن الاجابة على سؤالك
"""

In [25]:
question = input("Question: ")
grounding = proximity_search(question)
prompt = f"""عنوان الدرس والمادة:
{lessonInfo} \n {prompt_input1} \n{grounding} \n {prompt_input2} \n :السؤال هو \n {question}"""
print(f"Prompt: {prompt}")
generated_response = model_1.generate_text(prompt=prompt, guardrails=False)
print(f"ALLAM: {generated_response}")

Prompt: عنوان الدرس والمادة:
الوحدة الأولى الخلفاء الراشدون / الدرس الاول الخلفاء الراشدون - الدراسات الاجتماعية - خامس ابتدائي 
 انت مساعد ذكاء اصطناعي ومهمتك هي استخدام النص المرفق ادناه للرد على الاسئلة المطروحة 
المنورة اما اختيار عمر بن الخطاب رضي الله عنه فكان بتزكية من ابي بكر الصديق رضي الله عنهما قبل وفاته ورضاء جميع المسلمين واجماعهم على تولي الخليفة عمر بن الخطاب وكان اختيار عثمان بن عفان رضي الله عنه عن توافق وتشاور مشاورة بين جميع المسلمين وبين الصحابة وكان ايضا كذلك اختيار علي بن ابي طالب باجماع الصحابة وعموم المسلمين على توليهما الخلافة طبعا للاطلاع عندنا معلومة جميلة لنتطلع عليها ونتعرف عليها الخلفاء الراشدون هم الصحابة الذين تولوا بعد النبي
بن عفان واستمرت خلافته خمس سنوات وفي عام 40 للهجرة انتهت مدة الخلافة الراشدة وبدات دول اسلامية جديدة من فضائل الخلافة الخلفاء الراشدين وفضائل الخلافة الراشدين ما هي يا ترى من اول من اسلم بالنبي صلى الله عليه وسلم وصدق دعوة نعم هم الخلفاء الراشدين هم افضل الصحابة لا اختلاف عليهم ايضا من اواؤل من دخلوا الاسلام وساندوه بقوة ساندوا الاس